Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Potemkin Viktor"
COLLABORATORS = ""

---

In [ ]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [ ]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # YOUR CODE HERE
    norm = lambda v: np.sqrt(np.sum(v**2))
    outvec = np.zeros_like(vec)
    outvec[0] = norm(vec)
    if vec[0] > 0:
        d = vec.copy()
        d[0] = -np.sum(vec[1:]**2) / (vec[0] + norm(vec)) #(x_1 - |x|)
        u = d / norm(d)
    else:
        u = (vec - outvec) / norm(vec - outvec)
    u = np.reshape(u, (1, vec.shape[0])) 
    H = np.eye(vec.shape[0]) - 2 * u.T @ u

    return outvec, H

Test your function using tests below

In [ ]:
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(h @ v1, v, atol=1e-14)
assert_allclose(h @ v, v1, atol=1e-14)

assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(3), atol=1e-14)


# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, consider a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n$ times, we obtain

$$
\mathbf{H}_{n} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function which receives a rectangular matrix, $A$, and returns the $Q$ and $R$ factors of the $QR$ factorization of $A$.

In [ ]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # YOUR CODE HERE
    q = np.eye(m)
    for i in range(min(n,m)):
        h = np.eye(m)
        _, H = householder(a1[i:,i])
        h[i:, i:] = H
        a1 = h @ a1
        q = q @ h
    return q, a1

    #a = np.random.random(size=(3, 5))
    #q, r = qr_decomp(a)
    #np.assert_allclose(np.dot(q, r), a)
    

In [ ]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc
np.set_printoptions(suppress=True)

In [ ]:
rndm = np.random.default_rng(1234)
a = rndm.uniform(size=(5, 3))
aa = a.copy()

q, r = qr_decomp(a)

# check that `qr_decomp` leaves `a` intact
assert_allclose(a, aa,
                atol=1e-16)

# check that Q is orthogonal
assert_allclose(q @ q.T, np.eye(5),
                atol=1e-14)

# check if R is upper triangular
assert_allclose(np.triu(r), r,
                atol=1e-14)

# check the factorization
assert_allclose(q @ r, a,
                atol=1e-14)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(qq @ rr, a,
                atol=1e-14)

In [ ]:
q

array([[ 0.70927665, -0.65820276,  0.15769261, -0.13401225, -0.14446287],
       [ 0.19004031,  0.24683769, -0.27215068,  0.36996973, -0.83187273],
       [ 0.17557001,  0.27427984,  0.81623064,  0.47278277,  0.06472814],
       [ 0.19146175,  0.48153053,  0.2654666 , -0.77522383, -0.24500254],
       [ 0.62715938,  0.44580174, -0.40541593,  0.14376223,  0.47212526]])

In [ ]:
qq

array([[-0.70927665,  0.65820276, -0.15769261,  0.12555944, -0.15186715],
       [-0.19004031, -0.24683769,  0.27215068, -0.41678788, -0.80943048],
       [-0.17557001, -0.27427984, -0.81623064, -0.46832428,  0.09157321],
       [-0.19146175, -0.48153053, -0.2654666 ,  0.75999727, -0.2887947 ],
       [-0.62715938, -0.44580174,  0.40541593, -0.11661564,  0.47955254]])

In [ ]:
r

array([[ 1.37703641,  1.01237979,  1.38656757],
       [ 0.        ,  0.51330932,  0.28042833],
       [-0.        , -0.        ,  0.78865421],
       [ 0.        ,  0.        , -0.        ],
       [-0.        , -0.        , -0.        ]])

In [ ]:
rr

array([[-1.37703641, -1.01237979, -1.38656757],
       [ 0.        , -0.51330932, -0.28042833],
       [ 0.        ,  0.        , -0.78865421],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

Check if your `q` and `r` agree with `qq` and `rr`. Explain.
(Write up your explanations in this cell.)




# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$ themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function which uses reflection vectors (from the previous function) to multiply an input matrix with $\mathbf{Q}^T$. Make sure you properly comment your code.

In [ ]:
def qr_nomatrix(a):
    """Form QR decomposition of `a` via successive Householder reflections.
    
    Parameters
    ----------
    a : ndarray
        Input matrix
    
    Returns
    -------
    R : ndarray
        Upper triangular matrix of the QR decomposition
    
    U : ndarray
        Columns store successive Householder reflectors: `U[j:, j]` stores
        the Householder reflector for reducing the `j`-th column.
        
    See Also
    --------
    householder_apply : apply Householder reflectors stored in `U` to `a`.
    
    """
    # YOUR CODE HERE
    
    R = np.array(a, copy=True, dtype=float)
    m, n = R.shape
    norm = lambda v: np.sqrt(np.sum(v**2))

    U = np.zeros((m, min(m,n)))
    for i in range(min(m,n)):
        v = R[i:, i]
        y = np.zeros_like(v)
        y[0] = norm(v)
        if v[0] > 0:
            d = v.copy()
            d[0] = -np.sum(v[1:]**2) / (v[0] + norm(v))
            u = d / norm(d)
        else:
            u = (v - y) / norm(v-y)
        u = np.reshape(u, (1, v.shape[0]))
        for k in range(m - i):
            U[i:,i] = u
        u_ = np.reshape(U[:, i], (1, m))
        R = R - 2* u_.T @ (u_ @ R)
    return R, U

    

    
def householder_apply(b, uu):
    """Apply the Householder reflectors stored in `uu` to `b`.
    
    The result is equivalent to
    >>> r, q = qr_decomp(a)
    >>> q.T @ b
    
    Parameters
    ----------
    b : ndarray
        Input matrix
    uu : ndarray
        Householder reflectors: `U[j:, j]` is the reflection vector
        for transforming the `j`-th column of `a`.
        
    Returns
    -------
    r : ndarray
        The result of applying the reflectors to `b`. Equivalent to
        computing `q.T @ b`.

    See Also
    --------
    qr_decomp
    
    """
    # YOUR CODE HERE
    r = np.array(b, copy=True, dtype=float)
    for i in range(uu.shape[1]):
        u = np.reshape(uu[:, i], (1, uu.shape[0]))
        r = r - 2* u.T @ (u @ r)
    return r
    

In [ ]:
rndm = np.random.default_rng(1223)

a = rndm.uniform(size=(5, 5))
a_copy = a.copy()

R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
assert_allclose(R1, R2, atol=1e-14)
assert_allclose(a, a_copy, atol=1e-15)

from scipy.linalg import qr
R_lib = qr(a)[1]


In [ ]:
# Check consistency with the scipy library decomposition. Allow for sign differences

conds = [np.allclose(R2[i, :], R_lib[i, :], atol=1e-14) or
         np.allclose(R2[i, :], -R_lib[i, :], atol=1e-14) for i in range(5)]
assert False not in conds


In [ ]:
# More testing here, keep this cell intact
